In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📊 Growth Accounting: Decomposing Output Growth

Growth accounting provides a framework for understanding the sources of economic growth. It allows us to break down the growth rate of total output ($g_Y$) into contributions from the growth in inputs (capital $g_K$ and labor $g_L$) and the growth in overall efficiency or technology, known as Total Factor Productivity ($g_A$).

Assuming a standard Cobb-Douglas production function with constant returns to scale ($Y = A K^{\alpha} L^{1 - \alpha}$), the fundamental growth accounting equation is:

$$g_Y \approx \alpha g_K + (1 - \alpha) g_L + g_A$$

Where:
- $g_Y$: Growth rate of output ($Y$)
- $g_K$: Growth rate of capital ($K$)
- $g_L$: Growth rate of labor ($L$)
- $g_A$: Growth rate of Total Factor Productivity (TFP, $A$)
- $\alpha$: Capital's share of income (output elasticity of capital)
- $(1-\alpha)$: Labor's share of income (output elasticity of labor)

This equation states that output growth is approximately the sum of TFP growth and the weighted sum of input growth rates, where the weights are the respective income shares. TFP growth ($g_A$) captures the residual part of output growth that cannot be explained by the growth in measured inputs (K and L).

# 💡 Interpreting the Decomposition

The interactive simulation below calculates the total output growth ($g_Y$) based on the input growth rates ($g_K, g_L, g_A$) and the capital share ($\alpha$) provided via the sliders.

It then visualizes the decomposition in two ways:

1.  **Bar Chart:** Shows the absolute contribution of each component ($g_A$, $\alpha g_K$, and $(1-\alpha) g_L$) to the total output growth rate ($g_Y$).
2.  **Pie Chart:** Shows the *percentage share* of total output growth ($g_Y$) that is attributable to TFP growth, capital growth, and labor growth, respectively. This helps visualize the relative importance of each source.

By adjusting the sliders, you can explore scenarios such as:
* Growth driven primarily by TFP (technological progress).
* Growth driven by capital accumulation (investment).
* Growth driven by increases in the labor force.
* How changes in capital's share ($\alpha$) affect the contributions of capital and labor.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def growth_accounting_decomposition(alpha=0.33, gA=0.02, gK=0.04, gL=0.01):
    """
    Calculates and visualizes the growth accounting decomposition.

    Args:
        alpha (float): Capital share (0 < alpha < 1).
        gA (float): Growth rate of TFP (A).
        gK (float): Growth rate of Capital (K).
        gL (float): Growth rate of Labor (L).
    """
    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)

    # Calculate contributions
    contrib_A = gA
    contrib_K = alpha * gK
    contrib_L = (1 - alpha) * gL

    # Calculate total output growth
    gY = contrib_A + contrib_K + contrib_L

    # Prepare data for plots
    components = ['TFP (gA)', 'Capital (alpha*gK)', 'Labor ((1-alpha)*gL)']
    values = [contrib_A, contrib_K, contrib_L]
    # Ensure values are non-negative for pie chart representation if gY is positive
    # (Handle negative growth cases gracefully for pie chart)
    pie_values = np.maximum(values, 0) if gY > 0 else [1, 1, 1] # Avoid errors in pie chart for negative gY
    pie_labels = [f"{comp}\n({val/gY:.1%})" if gY > 1e-9 else comp for comp, val in zip(components, values)] # Show percentages if gY > 0
    colors = ['sandybrown', 'cornflowerblue', 'mediumseagreen'] # Matched colors from previous notebook

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 6)) # 1 row, 2 columns

    # Plot 1: Bar Chart of Contributions
    bars = axes[0].bar(components, values, color=colors, edgecolor='black')
    axes[0].axhline(gY, linestyle='--', color='black', label=f'Total Growth gY = {gY:.2%}')
    axes[0].set_ylabel("Contribution to Growth Rate")
    axes[0].set_title("Growth Accounting: Absolute Contributions")
    axes[0].legend()
    axes[0].grid(axis='y', linestyle='--', alpha=0.7)
    # Add text labels to bars
    for bar in bars:
        yval = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.2%}', va='bottom' if yval >= 0 else 'top', ha='center', fontsize=10)
    # Format y-axis as percentage
    axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))


    # Plot 2: Pie Chart of Shares (only meaningful if gY > 0)
    if gY > 1e-9:
        axes[1].pie(pie_values, labels=pie_labels, colors=colors, autopct='%1.1f%%',
                    startangle=90, wedgeprops={'edgecolor': 'black'})
        axes[1].set_title(f"Share of Total Growth (gY = {gY:.2%})")
    else:
        axes[1].text(0.5, 0.5, f"Total Growth gY = {gY:.2%}\n(Pie chart not meaningful)",
                     ha='center', va='center', fontsize=12)
        axes[1].set_title("Share of Total Growth")
        axes[1].set_xticks([])
        axes[1].set_yticks([])


    fig.suptitle("Growth Accounting Decomposition", fontsize=16, y=1.02)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Display Summary Equation ---
    summary_md = f"""
    ### 🧮 Growth Decomposition Equation:

    $g_Y \\approx g_A + \\alpha \\cdot g_K + (1 - \\alpha) \\cdot g_L$

    ${gY:.3f} \\approx {gA:.3f} + ({alpha:.2f} \\times {gK:.3f}) + ({1 - alpha:.2f} \\times {gL:.3f})$

    ${gY:.3f} \\approx {gA:.3f} + {contrib_K:.3f} + {contrib_L:.3f}$
    """
    display(Markdown(summary_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
interact(
    growth_accounting_decomposition,
    alpha=FloatSlider(value=0.33, min=0.1, max=0.9, step=0.01, description='Capital Share (alpha α):', style=style, readout_format='.2f'),
    gA=FloatSlider(value=0.015, min=-0.01, max=0.05, step=0.001, description='TFP Growth (gA):', style=style, readout_format='.1%'),
    gK=FloatSlider(value=0.04, min=0.0, max=0.1, step=0.005, description='Capital Growth (gK):', style=style, readout_format='.1%'),
    gL=FloatSlider(value=0.01, min=-0.01, max=0.05, step=0.005, description='Labor Growth (gL):', style=style, readout_format='.1%')
);


interactive(children=(FloatSlider(value=0.33, description='Capital Share (alpha α):', max=0.9, min=0.1, step=0…

# 📘 Theory Summary

The growth accounting equation:
\[
g_Y = g_A + \alpha \cdot g_K + (1 - \alpha) \cdot g_L
\]

- **gA**: Total Factor Productivity growth (technology)
- **gK**: Capital stock growth
- **gL**: Labor force growth
- **α**: Capital share in output

Use this model to explore whether a country’s growth is driven by:
- TFP (innovation)
- Capital deepening
- Labor expansion

**Sources**:
- GrowthEcon Study Guide [Ch. 6](https://growthecon.com/StudyGuide/growth-accounting.html)
- Charles I. Jones, *Macroeconomics*, Ch. 6